## Configuration
_Initial steps to get the notebook ready to play nice with our repository. Do not delete this section._

Code formatting with [black](https://pypi.org/project/nb-black/).

In [1]:
%load_ext lab_black

In [2]:
import os
import pathlib

In [3]:
this_dir = pathlib.Path(os.path.abspath(""))

In [4]:
data_dir = this_dir / "data"

In [5]:
import pytz
import glob
import requests
import pandas as pd
from slugify import slugify
from datetime import datetime

## Download

Retrieve the page

In [6]:
url = "https://services7.arcgis.com/RomaVqqozKczDNgd/ArcGIS/rest/services/C-19_IC_Cases_by_ZipCode_-_031722/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pjson&token="

In [7]:
r = requests.get(url)

In [8]:
data = r.json()

## Parse

In [9]:
dict_list = []

In [10]:
for item in data["features"]:
    d = dict(
        county="Imperial",
        area_name=item["attributes"]["CITY"],
        zip_code=item["attributes"]["ZIPCODE"],
        confirmed_cases=item["attributes"]["TOTAL_CASES"],
    )
    dict_list.append(d)

KeyError: 'features'

In [11]:
df = pd.DataFrame(dict_list)

In [12]:
df

county        area_name zip_code  confirmed_cases
0   Imperial          UNKNOWN  UNKNOWN              419
1   Imperial     OCOTILLO, CA    92004                0
2   Imperial         BARD, CA    92222               13
3   Imperial      BRAWLEY, CA    92227             9221
4   Imperial     CALEXICO, CA    92231            13583
5   Imperial   CALIPATRIA, CA    92233             1085
6   Imperial    EL CENTRO, CA    92243            13886
7   Imperial    EL CENTRO, CA    92244              226
8   Imperial        HEBER, CA    92249             2126
9   Imperial    HOLTVILLE, CA    92250             2038
10  Imperial     IMPERIAL, CA    92251             6478
11  Imperial       NILAND, CA    92257              318
12  Imperial     OCOTILLO, CA    92259               28
13  Imperial   PALO VERDE, CA    92266               15
14  Imperial       SEELEY, CA    92273              409
15  Imperial      THERMAL, CA    92274             1053
16  Imperial  SALTON CITY, CA    92275               65
17  Imperial  WESTMORLAND, CA    92281              590
18  Imperial  WINTERHAVEN, CA    92283              807

In [13]:
df["area_name"] = df["area_name"].str.replace(", CA", "").str.title()

Match zips to names

In [14]:
df["area_name"] = df["zip_code"] + ": " + df["area_name"]

In [15]:
df.loc[(df.zip_code == "92004"), "area_name"] = "92004: Borrego Springs"

In [16]:
df.loc[(df.zip_code == "UNKNOWN"), "area_name"] = "Unknown"

Get timestamp

In [20]:
date_url = "https://services7.arcgis.com/RomaVqqozKczDNgd/ArcGIS/rest/services/C-19_IC_Cases_by_ZipCode_-_031722/FeatureServer/0/?f=json"
date_r = requests.get(date_url)
date_data = date_r.json()

In [21]:
timestamp = date_data["editingInfo"]["lastEditDate"]

In [22]:
timestamp = datetime.fromtimestamp((timestamp / 1000))

In [23]:
latest_date = pd.to_datetime(timestamp).date()

In [24]:
df["county_date"] = latest_date

In [25]:
export_df = df[
    ["county", "area_name", "confirmed_cases", "county_date", "zip_code"]
].rename(columns={"area_name": "city", "zip_code": "zip"})

## Vet

In [27]:
try:
    assert not len(export_df) < 19
except AssertionError:
    raise AssertionError("Imperial County's scraper is missing rows")

In [28]:
try:
    assert not len(export_df) > 19
except AssertionError:
    raise AssertionError("Imperial County's scraper has more rows than before")

## Export

In [29]:
tz = pytz.timezone("America/Los_Angeles")

In [30]:
today = datetime.now(tz).date()

In [31]:
slug = "imperial"

In [32]:
df.to_csv(data_dir / slug / f"{today}.csv", index=False)

## Combine

In [33]:
csv_list = [
    i
    for i in glob.glob(str(data_dir / slug / "*.csv"))
    if not str(i).endswith("timeseries.csv")
]

In [34]:
df_list = []
for csv in csv_list:
    if "manual" in csv:
        df = pd.read_csv(csv, parse_dates=["date"])
    else:
        file_date = csv.split("/")[-1].replace(".csv", "")
        df = pd.read_csv(csv, parse_dates=["county_date"]).rename(
            columns={"area_name": "area", "city": "area"}
        )
        df["date"] = file_date
    df_list.append(df)

In [35]:
df = pd.concat(df_list).sort_values(["date", "area"])

In [36]:
df.to_csv(data_dir / slug / "timeseries.csv", index=False)